**Data Description:**
- Player Database Dataset
- 196 Observations
- The average age is 21.14 years
- The average hours played is 5.85 hours


ISSUES:
- NA values in the gender variable
- Very large outliers, and a lot of 0 hours played entries


7 Variables:
- experience - String, and is one of: Beginner, Amateur, Regular, Pro, Veteran - Describes the level of experience a player has with the game
- subscribe - Boolean - Describes whether or not the player is subscribed to the newsletter
- hashedEmail - String - Player's email encrypted
- played_hours - Double - Player's number of hours on the server
- name - String - Player's real name
- gender - String - Player's gender
- Age - Integer - Player's age in years